<a href="https://colab.research.google.com/github/Vonewman/Hugginface-course/blob/main/Biais_et_limitations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 27.8 MB/s 
     |████████████████████████████████| 596 kB 47.2 MB/s 
     |████████████████████████████████| 101 kB 10.9 MB/s 
     |████████████████████████████████| 6.6 MB 50.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# Biais et limitations

Si vous souhaitez utiliser un modèle pré-entraîné ou une version finetunée de celui-ci en production, il est important d’avoir conscience que, bien que ces modèles soient puissants, ils ont des limites. La plus importante de ces limitations est que, pour permettre le pré-entraînement des modèles sur de grandes quantités de données, les chercheurs récupèrent souvent tout le contenu qu’ils peuvent trouver et donc en prenant le meilleur et le pire de ce qui est disponible sur internet.

Pour illustrer cela rapidement, revenons au pipeline fill-mask avec le modèle BERT :

In [4]:
from transformers import pipeline

unmasker = pipeline("fill-mask", model="bert-base-uncased")
result = unmasker("This man works as a [MASK].")  # Cet homme travaille comme [MASQUE]
print([r["token_str"] for r in result])

result = unmasker("This woman works as a [MASK].")  # Cette femme travaille comme [MASQUE]
print([r["token_str"] for r in result])

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


['carpenter', 'lawyer', 'farmer', 'businessman', 'doctor']
['nurse', 'maid', 'teacher', 'waitress', 'prostitute']


In [5]:
['lawyer', 'carpenter', 'doctor', 'waiter', 'mechanic'] 
# [avocat, charpentier, médecin, serveur, mécanicien]
['nurse', 'waitress', 'teacher', 'maid', 'prostitute'] 
# ["infirmière", "serveuse", "professeur", "femme de chambre", "prostituée"]    

['nurse', 'waitress', 'teacher', 'maid', 'prostitute']

Lorsque l’on demande au modèle de remplacer le mot manquant dans ces deux phrases, il ne propose qu’un seul métier ne portant pas la marque du genre (waiter/waitress → serveur/serveuse). Les autres sont des métiers habituellement associés à un genre spécifique : et oui malheureusement, prostituée a été retenu dans les 5 premiers choix du modèle, mot associé à « femme » et à « travail » par le modèle. Cela se produit même si BERT est l’un des rare transformers qui n’a pas été construit avec des données récupérées par scrapping sur internet, mais à l’aide de données en apparence neutres. En effet, il est entraîné sur les jeux de donnés Wikipédia Anglais et BookCorpus).

Donc lorsque vous utilisez ce genre d’outils, il est important de garder en tête que le modèle que vous utilisez peut rapidement générer du contenu sexiste, raciste ou homophobe. Le finetuning du modèle sur vos données ne fera en aucun cas disparaître ce biais intrinsèque.